In [22]:
import json

import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None

In [23]:
df_elections = pd.read_csv('/mnt/data/_results/asset_evolution_elections_2018_2.csv')

In [24]:
# Filtra por últimas eleições
df_elections = df_elections[df_elections.year.isin([2010, 2014, 2018])]

In [25]:
# Selectiona candidatos que concorreram ao menos em 2 eleições
counts = df_elections.name.value_counts()
candidates = counts[counts>=2].index

In [26]:
# Separa o dataframe apenas usando candidatos de ao menos 2 eleições
df = df_elections[df_elections.name.isin(candidates)]
df.head(2)

,name,num_of_elections_run,year,state,party,position,result,sum_of_assets_declared,num_of_assets_declared,Unnamed: 9
412752,ABDALA HABIB FRAXE JUNIOR,8,2018,AM,PODE,DEPUTADO(A) ESTADUAL,NÃO INFORMADO,550000.0,1.0,NaN
412752,ABDALA HABIB FRAXE JUNIOR,8,2014,AM,PODE,VICE-GOVERNADOR(A),NÃO ELEITO,550000.0,1.0,NaN


In [27]:
df.shape

(6183, 10)

In [28]:
# Remove lugares onde temos None
df.sum_of_assets_declared = df.sum_of_assets_declared.str.replace('None', '0')

In [29]:
# Substitui valores ausentes por 0
df.sum_of_assets_declared.fillna(0, inplace=True)

# Muda coluna de soma de bens para decimal
df.sum_of_assets_declared = df.sum_of_assets_declared.astype(float)

In [30]:
# Transformando em milhoes para facilitar
df.sum_of_assets_declared = df.sum_of_assets_declared / 10**6

In [31]:
df.head()

,name,num_of_elections_run,year,state,party,position,result,sum_of_assets_declared,num_of_assets_declared,Unnamed: 9
412752,ABDALA HABIB FRAXE JUNIOR,8,2018,AM,PODE,DEPUTADO(A) ESTADUAL,NÃO INFORMADO,0.550000,1.0,NaN
412752,ABDALA HABIB FRAXE JUNIOR,8,2014,AM,PODE,VICE-GOVERNADOR(A),NÃO ELEITO,0.550000,1.0,NaN
412752,ABDALA HABIB FRAXE JUNIOR,8,2014,AM,PTN,DEPUTADO(A) ESTADUAL,ELEITO POR QP,0.550000,1.0,NaN
412752,ABDALA HABIB FRAXE JUNIOR,8,2010,AM,PTN,DEPUTADO(A) ESTADUAL,ELEITO,0.550000,1.0,NaN
262151,ABEILARD GOULART DE SOUZA FILHO,12,2018,RJ,PDT,DEPUTADO(A) FEDERAL,NÃO INFORMADO,0.152298,4.0,NaN


# Análise por pessoa

In [32]:
df.position = pd.Categorical(df.position,
                             categories=['PRESIDENTE(A) DA REPÚBLICA',  'GOVERNADOR(A)', 'SENADOR(A)', 
                                         'DEPUTADO(A) FEDERAL', 'DEPUTADO(A) ESTADUAL',
                                         'VICE-GOVERNADOR(A)', '2º SUPLENTE', 
                                         '1º SUPLENTE SENADOR(A)', '1º SUPLENTE', '2º SUPLENTE SENADOR(A)', 
                                         'VICE-PRESIDENTE(A)', 'DEPUTADO(A) DISTRITAL'][::-1],
                             ordered=True)

In [33]:
df.result = pd.Categorical(df.result, 
                           categories=['ELEITO', 'ELEITO POR QP', 'NÃO ELEITO', 
                                       'REJEITADO', 'SUPLENTE', 'SUBSTITUÍDO', 
                                       '2o TURNO','0'][::-1],
                           ordered=True)

In [34]:
df_result = df.groupby(['name', 'year'])['result'].max().unstack().reset_index()
df_result.rename(columns={
    2010.0: '2010-resultado',
    2014.0: '2014-resultado',
    2018.0: '2018-resultado',
}, inplace=True)

del df_result.columns.name
df_result.set_index('name', inplace=True)
df_result.reset_index()

df_result.drop(['2018-resultado'], axis=True, inplace=True)
df_result.head()

,2010-resultado,2014-resultado
name,,
ABDALA HABIB FRAXE JUNIOR,ELEITO,ELEITO POR QP
ABEILARD GOULART DE SOUZA FILHO,NaN,SUPLENTE
ABEL DO PORTO SANTOS,NÃO ELEITO,NaN
ABEL MOACIR BURGDURFF DE MORAES,NaN,NÃO ELEITO
ABIDENE SALUSTIANO DA SILVA,NaN,SUPLENTE


In [35]:
df_position = df.groupby(['name', 'year'])['position'].max().unstack().reset_index()
df_position.rename(columns={
    2010.0: '2010-cargo',
    2014.0: '2014-cargo',
    2018.0: '2018-cargo',
}, inplace=True)

del df_position.columns.name
df_position.set_index('name', inplace=True)
df_position.reset_index()
df_position.head()

,2010-cargo,2014-cargo,2018-cargo
name,,,
ABDALA HABIB FRAXE JUNIOR,DEPUTADO(A) ESTADUAL,DEPUTADO(A) ESTADUAL,DEPUTADO(A) ESTADUAL
ABEILARD GOULART DE SOUZA FILHO,NaN,DEPUTADO(A) ESTADUAL,DEPUTADO(A) FEDERAL
ABEL DO PORTO SANTOS,DEPUTADO(A) FEDERAL,NaN,DEPUTADO(A) FEDERAL
ABEL MOACIR BURGDURFF DE MORAES,NaN,1º SUPLENTE,DEPUTADO(A) ESTADUAL
ABIDENE SALUSTIANO DA SILVA,NaN,DEPUTADO(A) FEDERAL,DEPUTADO(A) ESTADUAL


In [36]:
df_assets = df.groupby(['name', 'year'])['sum_of_assets_declared'].max().unstack().reset_index()
df_assets.rename(columns={
    2010.0: '2010-soma-bens',
    2014.0: '2014-soma-bens',
    2018.0: '2018-soma-bens',
}, inplace=True)


del df_assets.columns.name
df_assets.set_index('name', inplace=True)
df_assets.reset_index()
df_assets.head()

,2010-soma-bens,2014-soma-bens,2018-soma-bens
name,,,
ABDALA HABIB FRAXE JUNIOR,0.550,0.550000,0.550000
ABEILARD GOULART DE SOUZA FILHO,NaN,0.112152,0.152298
ABEL DO PORTO SANTOS,0.035,NaN,0.025000
ABEL MOACIR BURGDURFF DE MORAES,NaN,0.005000,0.303000
ABIDENE SALUSTIANO DA SILVA,NaN,0.028000,0.135000


In [37]:
dd = pd.concat([df_result, df_position, df_assets], axis=1)

dd['2010-2014'] = dd['2014-soma-bens'] - dd['2010-soma-bens']
dd['2010-2014-percent'] = ((dd['2014-soma-bens'] - dd['2010-soma-bens']) / dd['2010-soma-bens'])*100
dd['2014-2018'] = dd['2018-soma-bens'] - dd['2014-soma-bens']
dd['2014-2018-percent'] = ((dd['2018-soma-bens'] - dd['2014-soma-bens']) / dd['2014-soma-bens'])*100
dd['2010-2018'] = dd['2018-soma-bens'] - dd['2010-soma-bens']
dd['2010-2018-percent'] = ((dd['2018-soma-bens'] - dd['2010-soma-bens']) / dd['2010-soma-bens'])*100


#dd['2010-2014-percent-pretty'] = dd['2010-2014-percent'].apply(lambda x: '{:.1f}%'.format(x))
dd['2014-2018-percent-pretty'] = dd['2014-2018-percent'].apply(lambda x: '{:.1f}%'.format(x))
dd['2010-2018-percent-pretty'] = dd['2010-2018-percent'].apply(lambda x: '{:.1f}%'.format(x))

df_final = dd[[
    #'2010-cargo', '2010-resultado', '2010-soma-bens', 
    '2014-cargo', '2014-resultado', '2014-soma-bens', #'2010-2014', '2010-2014-percent', '2010-2014-percent-pretty',
    '2018-cargo', '2018-soma-bens', '2014-2018', '2014-2018-percent', '2014-2018-percent-pretty', 
    #'2010-2018', '2010-2018-percent', '2010-2018-percent-pretty',
]]

In [38]:
drop = [
    #'2010-2014-percent', 
    '2014-2018-percent', 
    #'2010-2018-percent'
]

In [39]:
top30_2018 = df_final.sort_values(by='2018-soma-bens', ascending=False).head(30)
top30_2018.drop(drop, axis=1, inplace=True)
top30_2018.to_csv('/mnt/data/_results/top-30-ricos-nas-eleicoes-2018.csv')
top30_2018.head(10)

,2014-cargo,2014-resultado,2014-soma-bens,2018-cargo,2018-soma-bens,2014-2018,2014-2018-percent-pretty
name,,,,,,,
OTAVIANO OLAVO PIVETTA,NaN,NaN,NaN,VICE-GOVERNADOR(A),379.456427,NaN,nan%
MARDEN LUIS BRITO CAVALCANTE E MENESES,DEPUTADO(A) ESTADUAL,ELEITO POR QP,0.619703,DEPUTADO(A) ESTADUAL,153.719389,153.099686,24705.3%
PEDRO CHAVES DOS SANTOS FILHO,NaN,NaN,NaN,SENADOR(A),130.494561,NaN,nan%
MAURO MENDES FERREIRA,NaN,NaN,NaN,GOVERNADOR(A),113.453807,NaN,nan%
IVONE MEZZOMO CASSOL,SENADOR(A),NÃO ELEITO,56.276546,1º SUPLENTE,84.714269,28.437724,50.5%
NILSO JOSE BERLANDA,DEPUTADO(A) ESTADUAL,SUPLENTE,6.266501,DEPUTADO(A) ESTADUAL,64.809477,58.542976,934.2%
REINALDO AZAMBUJA SILVA,GOVERNADOR(A),ELEITO,27.440416,GOVERNADOR(A),38.698697,11.258282,41.0%
DAMINA DE CARVALHO PEREIRA,DEPUTADO(A) FEDERAL,ELEITO POR QP,38.827718,DEPUTADO(A) FEDERAL,37.586761,-1.240957,-3.2%
EDENILSO ROSSI ARNALDI,DEPUTADO(A) FEDERAL,NaN,27.077399,DEPUTADO(A) FEDERAL,33.589670,6.512271,24.1%


In [40]:
top30_crescimento_absoluto_2018 = df_final.sort_values(by='2014-2018', ascending=False).head(30)
top30_crescimento_absoluto_2018.drop(drop, axis=1, inplace=True)
top30_crescimento_absoluto_2018.to_csv(
    '/mnt/data/_results/top-30-que-enriqueceram-desde-as-eleicoes-2014-valor-absoluto.csv')
top30_crescimento_absoluto_2018.head(10)

,2014-cargo,2014-resultado,2014-soma-bens,2018-cargo,2018-soma-bens,2014-2018,2014-2018-percent-pretty
name,,,,,,,
MARDEN LUIS BRITO CAVALCANTE E MENESES,DEPUTADO(A) ESTADUAL,ELEITO POR QP,0.619703,DEPUTADO(A) ESTADUAL,153.719389,153.099686,24705.3%
NILSO JOSE BERLANDA,DEPUTADO(A) ESTADUAL,SUPLENTE,6.266501,DEPUTADO(A) ESTADUAL,64.809477,58.542976,934.2%
IVONE MEZZOMO CASSOL,SENADOR(A),NÃO ELEITO,56.276546,1º SUPLENTE,84.714269,28.437724,50.5%
DILMAR DAL BOSCO,DEPUTADO(A) ESTADUAL,ELEITO POR QP,2.394865,DEPUTADO(A) ESTADUAL,22.252168,19.857304,829.2%
MARINALDO ROSENDO DE ALBUQUERQUE,DEPUTADO(A) FEDERAL,ELEITO POR QP,13.026175,DEPUTADO(A) FEDERAL,32.235112,19.208937,147.5%
MIGUEL MOUBADDA HADDAD,DEPUTADO(A) FEDERAL,ELEITO POR QP,11.383318,DEPUTADO(A) FEDERAL,29.316575,17.933257,157.5%
ALFREDO COTAIT NETO,2º SUPLENTE,NÃO ELEITO,4.746242,1º SUPLENTE,16.050417,11.304175,238.2%
REINALDO AZAMBUJA SILVA,GOVERNADOR(A),ELEITO,27.440416,GOVERNADOR(A),38.698697,11.258282,41.0%
ANTONIO JORGE GODINHO,DEPUTADO(A) ESTADUAL,SUPLENTE,1.649338,SENADOR(A),12.261525,10.612187,643.4%


In [41]:
df_sem_percent_infinito = df_final[df_final['2014-2018-percent']!= np.inf]
top30_crescimento_percent_2018 = df_sem_percent_infinito.sort_values(by='2014-2018-percent', ascending=False).head(30)
top30_crescimento_percent_2018.drop(drop, axis=1, inplace=True)
top30_crescimento_percent_2018.to_csv(
    '/mnt/data/_results/top-30-que-enriqueceram-desde-as-eleicoes-2014-valor-percent.csv')

top30_crescimento_percent_2018.head(10)

,2014-cargo,2014-resultado,2014-soma-bens,2018-cargo,2018-soma-bens,2014-2018,2014-2018-percent-pretty
name,,,,,,,
RUBENS SANCHEZ PROENCA,DEPUTADO(A) FEDERAL,NaN,3.000000e-08,DEPUTADO(A) FEDERAL,0.012000,0.012000,39999900.0%
MARDEN LUIS BRITO CAVALCANTE E MENESES,DEPUTADO(A) ESTADUAL,ELEITO POR QP,6.197028e-01,DEPUTADO(A) ESTADUAL,153.719389,153.099686,24705.3%
CARLOS DO SOCORRO GUERREIRO VAZ,DEPUTADO(A) FEDERAL,SUPLENTE,3.000000e-03,DEPUTADO(A) FEDERAL,0.507000,0.504000,16800.0%
CLEINALDO CASTRO LOPES,DEPUTADO(A) ESTADUAL,SUPLENTE,3.500000e-03,DEPUTADO(A) ESTADUAL,0.369485,0.365985,10456.7%
NAIARA LOURENCO,DEPUTADO(A) DISTRITAL,NÃO ELEITO,2.000000e-02,DEPUTADO(A) FEDERAL,2.046000,2.026000,10130.0%
GUSTAVO DE BIASE NUNES DA ROCHA,DEPUTADO(A) FEDERAL,SUPLENTE,2.500000e-03,DEPUTADO(A) FEDERAL,0.171000,0.168500,6740.0%
BESALIEL DE OLIVEIRA RODRIGUES,DEPUTADO(A) ESTADUAL,NÃO ELEITO,1.300000e-02,DEPUTADO(A) ESTADUAL,0.880000,0.867000,6669.2%
ABEL MOACIR BURGDURFF DE MORAES,1º SUPLENTE,NÃO ELEITO,5.000000e-03,DEPUTADO(A) ESTADUAL,0.303000,0.298000,5960.0%
SILVIO LUIZ RODRIGUES RAMALHO,DEPUTADO(A) ESTADUAL,SUPLENTE,5.000000e-03,DEPUTADO(A) ESTADUAL,0.244000,0.239000,4780.0%


# Análise de Evolução Patrimonial por Partido

In [42]:
df.party = pd.Categorical(df.party,
                             categories=['PDT', 'PSD', 'PSTU', 'PV', 'PSDB', 'PMDB', 'PROS', 'PSOL', 'PP',
                                         'PRP', 'PSB', 'PT', 'PSC', 'PRB', 'PMN', 'PR', 'PPS', 'PTC',
                                         'PODE', 'PSL', 'PRTB', 'PTN', 'PSDC', 'DEM', 'PMB', 'PHS', 'SD',
                                         'REDE', 'PPL', 'PTB', 'PEN', 'PCB', 'NOVO', 'PCO'],
                             ordered=True)

In [43]:
df_evolution = dd[[
    '2010-2014', '2010-2014-percent',
    '2014-2018', '2014-2018-percent',
]]
df_evolution.head()

,2010-2014,2010-2014-percent,2014-2018,2014-2018-percent
name,,,,
ABDALA HABIB FRAXE JUNIOR,0.0,0.0,0.000000,0.000000
ABEILARD GOULART DE SOUZA FILHO,NaN,NaN,0.040146,35.795660
ABEL DO PORTO SANTOS,NaN,NaN,NaN,NaN
ABEL MOACIR BURGDURFF DE MORAES,NaN,NaN,0.298000,5960.000000
ABIDENE SALUSTIANO DA SILVA,NaN,NaN,0.107000,382.142857


In [44]:
df_party = df.groupby(['name', 'year'])['party'].max().unstack().reset_index()
df_party.rename(columns={
    2010.0: '2010-partido',
    2014.0: '2014-partido',
    2018.0: '2018-partido',
}, inplace=True)

del df_party.columns.name
df_party.set_index('name', inplace=True)
df_party.reset_index()

df_party.head()

,2010-partido,2014-partido,2018-partido
name,,,
ABDALA HABIB FRAXE JUNIOR,PTN,PTN,PODE
ABEILARD GOULART DE SOUZA FILHO,NaN,SD,PDT
ABEL DO PORTO SANTOS,PRTB,NaN,PRB
ABEL MOACIR BURGDURFF DE MORAES,NaN,PSTU,PSTU
ABIDENE SALUSTIANO DA SILVA,NaN,PSC,PSC


In [45]:
dd = pd.concat([df_evolution, df_party], axis=1, sort=False)
dd = dd[['2010-partido', '2010-2014', '2010-2014-percent', '2014-partido', '2014-2018', '2014-2018-percent']]
dd.head()

,2010-partido,2010-2014,2010-2014-percent,2014-partido,2014-2018,2014-2018-percent
name,,,,,,
ABDALA HABIB FRAXE JUNIOR,PTN,0.0,0.0,PTN,0.000000,0.000000
ABEILARD GOULART DE SOUZA FILHO,NaN,NaN,NaN,SD,0.040146,35.795660
ABEL DO PORTO SANTOS,PRTB,NaN,NaN,NaN,NaN,NaN
ABEL MOACIR BURGDURFF DE MORAES,NaN,NaN,NaN,PSTU,0.298000,5960.000000
ABIDENE SALUSTIANO DA SILVA,NaN,NaN,NaN,PSC,0.107000,382.142857


In [46]:
party_evolution_2010 = dd.groupby(['2010-partido'])['2010-2014'].sum()
party_count_2010 = dd.groupby(['2010-partido'])['2010-2014'].count()


evolution_2010_proportional = party_evolution_2010/party_count_2010 
evolution_2010_proportional_sorted = evolution_2010_proportional.sort_values(ascending=False)[0:10]
evolution_2010_proportional_sorted.to_csv('/mnt/data/_results/top-10-partidos-que-mais-enriqueceram-2010a2014.csv')

evolution_2010_proportional_sorted

2010-partido
PR      1.059006
DEM     0.762060
PSDB    0.604462
PSDC    0.584179
PP      0.581336
PMN     0.542390
PTB     0.521047
PSC     0.489934
PV      0.478101
PDT     0.364981
Name: 2010-2014, dtype: float64

In [47]:
party_evolution_2014 = dd.groupby(['2014-partido'])['2014-2018'].sum()
party_count_2014 = dd.groupby(['2014-partido'])['2014-2018'].count()

evolution_2014_proportional = party_evolution_2014/party_count_2014
evolution_2014_proportional_sorted = evolution_2014_proportional.sort_values(ascending=False)[0:10]
evolution_2014_proportional_sorted.to_csv('/mnt/data/_results/top-10-partidos-que-mais-enriqueceram-2014a2018.csv')

evolution_2014_proportional_sorted

2014-partido
DEM     1.563673
PSDB    0.916195
PSD     0.697548
PR      0.693890
PP      0.473686
PSB     0.434132
PMDB    0.416924
PROS    0.312837
PTN     0.267217
PDT     0.196840
Name: 2014-2018, dtype: float64

# Análise de Declaração Patrimonial por Partido

In [48]:
assets_2018 = pd.concat([df_party['2018-partido'], df_final['2018-soma-bens']], axis=1)

In [49]:
party_assets_sum = assets_2018.groupby(['2018-partido'])['2018-soma-bens'].sum()
party_person_count = assets_2018.groupby(['2018-partido'])['2018-soma-bens'].count()

party_declared_mean_asset_values = party_assets_sum / party_person_count
party_declared_mean_asset_values_ordered = party_declared_mean_asset_values.sort_values(ascending=False)[0:10]

party_declared_mean_asset_values_ordered.to_csv('/mnt/data/_results/top-10-partidos-que-ricos-eleicoes-2018.csv')
party_declared_mean_asset_values_ordered

2018-partido
PDT     3.974812
DEM     3.600192
PSDB    3.041721
PP      2.840747
NOVO    2.630915
PR      2.363240
PSD     2.146579
PRB     1.853996
PODE    1.134835
PSB     0.964551
Name: 2018-soma-bens, dtype: float64